<a href="https://colab.research.google.com/github/Lucia1009/TFG-Plataforma-para-la-prediccion-de-potencia-electrica-generada-en-parques-eolicos/blob/desarrollo/modelos/comparacion_predicciones_diarias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dash -q

In [2]:
from google.colab import drive

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle

from dash import Dash, html, dcc, Input, Output
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [3]:
drive.mount('/content/drive', force_remount=False)

reales=pd.read_json('/content/drive/MyDrive/TFG/datos santiago/RedMeteorológica_Santiago-EOAS_diezminutal_28-03-2025_to_06-04-2025.json')

#cargo las prediccines con pickle
preds_28=pickle.load(open('/content/drive/MyDrive/TFG/datos santiago/forecast_2025-03-28.pkl', 'rb'))
preds_29=pickle.load(open('/content/drive/MyDrive/TFG/datos santiago/forecast_2025-03-29.pkl', 'rb'))
preds_30=pickle.load(open('/content/drive/MyDrive/TFG/datos santiago/forecast_2025-03-30.pkl', 'rb'))
preds_31=pickle.load(open('/content/drive/MyDrive/TFG/datos santiago/forecast_2025-03-31.pkl', 'rb'))

preds_04=pickle.load(open('/content/drive/MyDrive/TFG/datos santiago/forecast_2025-04-04.pkl', 'rb'))
preds_05=pickle.load(open('/content/drive/MyDrive/TFG/datos santiago/forecast_2025-04-05.pkl', 'rb'))
preds_06=pickle.load(open('/content/drive/MyDrive/TFG/datos santiago/forecast_2025-04-06.pkl', 'rb'))

predicciones=[preds_28,preds_29,preds_30,preds_31,preds_04,preds_05,preds_06]

# Cambio el nombre de las columnas para todos los dicionarios
for preds in predicciones:
  for modelo in preds:
    preds[modelo] = pd.DataFrame(preds[modelo])
    preds[modelo].rename(columns={'date': 'Fecha', 'temperature_2m': 'temperatura', 'wind_speed_10m': 'wind_speed', 'wind_direction_10m':'wind_direction'}, inplace=True)

display(predicciones[0]['best_match'].head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Fecha,temperatura,wind_speed,wind_direction,wd_sin_10m,wd_cos_10m
0,2025-03-28 00:00:00+00:00,7.5825,7.412853,29.054508,0.485642,0.874158
1,2025-03-28 01:00:00+00:00,6.5825,8.217153,28.810698,0.481917,0.876217
2,2025-03-28 02:00:00+00:00,6.0825,7.903619,30.068504,0.501035,0.865427
3,2025-03-28 03:00:00+00:00,5.5325,7.787991,33.690102,0.554701,0.832050
4,2025-03-28 04:00:00+00:00,5.2825,7.993297,35.837746,0.585492,0.810678


In [4]:
"""Trato los datos reales de santiago"""

unique_codigos = reales['Código'].unique().tolist()
datos = pd.DataFrame(columns=['Fecha']+unique_codigos )

for codigo in unique_codigos:

  variable= reales[reales['Código'] ==codigo][['Fecha','Código','Valor', 'Código de validación']]
  variable=variable[variable['Código de validación']==1]

  datos['Fecha'] = reales['Fecha'].unique()

  merged_data = datos.merge(variable[['Fecha', 'Valor']], on='Fecha', how='left')
  datos[codigo] = merged_data['Valor']

datos['Fecha'] = pd.to_datetime(datos['Fecha'], format='%d/%m/%Y %H:%M')
datos['Fecha']=datos['Fecha'].dt.tz_localize(None)
datos.set_index('Fecha', inplace=True)


# display(datos.head(10))
print(datos.shape)

# Datos con la media de cada hora
reales=datos.resample('h').mean()

#renombramos columnas
reales = reales.rename(columns={'TA_AVG_1.5m': 'temperatura', 'VV_AVG_10m': 'wind_speed', 'DV_AVG_10m': 'wind_direction'})

display(reales.head(10))
print(reales.shape)

(1434, 3)


,wind_direction,temperatura,wind_speed
Fecha,,,
2025-03-28 00:00:00,30.200000,8.040000,16.200000
2025-03-28 01:00:00,36.333333,7.650000,15.683333
2025-03-28 02:00:00,36.500000,7.535000,16.766667
2025-03-28 03:00:00,31.000000,6.873333,17.233333
2025-03-28 04:00:00,25.000000,6.365000,18.716667
2025-03-28 05:00:00,41.166667,5.895000,13.650000
2025-03-28 06:00:00,36.166667,5.776667,14.300000
2025-03-28 07:00:00,36.333333,5.805000,11.950000
2025-03-28 08:00:00,45.333333,7.461667,11.766667


(241, 3)


In [8]:
"""Hago un gráfico con las predicciones y los datos reales"""

atribs=['temperatura', 'wind_speed', 'wind_direction']
lista_modelos=list(predicciones[0].keys())
#Lista de colores para los distintos días
colores=['blue', 'green', 'yellow', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']

app = Dash('app1')

app.layout = html.Div([

  dcc.Dropdown(lista_modelos, lista_modelos[0], id='modelo', clearable=False),
  dcc.Dropdown(atribs, atribs[0], id='atributo', clearable=False),

  dcc.Graph(id='graph')
])


@app.callback(
    Output('graph', 'figure'),
    Input('modelo', 'value'),
    Input('atributo', 'value'))
def update_output_div(modelo, atributo):

  fig = go.Figure()


  fig.add_trace(go.Scatter(y=reales[atributo], x=reales.index, mode='lines', marker=dict(color='red'), name='Historicas estacion'))


  for preds, color in zip(predicciones, colores):

    reales_cp = reales.copy()
    reales_cp=reales_cp.reset_index()

    df_pred = preds[modelo].copy()
    df_pred['Fecha'] = pd.to_datetime(df_pred['Fecha']).dt.tz_localize(None)


    datos_trabajo_model = reales_cp.merge(
        df_pred,
        on="Fecha",
        how="inner",
        suffixes=("", f"_{modelo}")
    )


    fig.add_trace(go.Scatter(
        y=datos_trabajo_model[f"{atributo}_{modelo}"],
        x=datos_trabajo_model["Fecha"],
        mode='lines',
        marker=dict(color=color),
        opacity=0.5,
        name=datos_trabajo_model["Fecha"][0].strftime('%d-%m-%Y')
    ))



  return fig

if __name__ == '__main__':
    app.run(debug=True, port=8081)

<IPython.core.display.Javascript object>

# Error por día para cada conjunto de datos